In [ ]:
import csv
import json
import os
import random
import sys
from tqdm import tqdm
import urllib.request

## --- parameter ---
CONTEST = "W404" # Contest name
CONTEST_URL = "https://leetcode.com/contest/weekly-contest-404/" # Contest URL
CONTEST_API_URL = "https://leetcode.com/contest/api/ranking/weekly-contest-404/" # API URL

headers1 = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version': '"126.0.6478.127"',
    'sec-ch-ua-full-version-list': '"Not/A)Brand";v="8.0.0.0", "Chromium";v="126.0.6478.127", "Google Chrome";v="126.0.6478.127"',
    'sec-ch-ua-platform-version': '"15.0.0"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
}

headers2 = headers1.copy()
headers2['referer'] = 'https://leetcode.com/contest/'

In [ ]:
user = []
Q3 = []
Q4 = []
Q = []

## get question_id
for _ in range(10):
    try:
        req = urllib.request.Request(CONTEST_API_URL, headers=random.choice([headers1, headers2]))
        r = urllib.request.urlopen(req).read()
        response = json.loads(r)
    except:
        pass
    else:
        break
else:
    print("http request error:", CONTEST_API_URL)
    sys.exit()

for question in response["questions"]:
    Q.append(str(question["question_id"]))
    print(question["question_id"], question["credit"], question["title"])

## get ranking page
PAGE = 500 # how many page gonna fetch
for pagination in tqdm(range(1, PAGE+1)):
    for _ in range(10):
        try:
            req = urllib.request.Request(CONTEST_API_URL+"?pagination=" + str(pagination) +"&region=global", headers=random.choice([headers1, headers2]))
            r = urllib.request.urlopen(req).read()
            response = json.loads(r)
        except:
            pass
        else:
            break
    else:
        print("http request error:", CONTEST_API_URL+"?pagination=" + str(pagination) +"&region=global")
        sys.exit()

    for i in range(25):
        user.append(response["total_rank"][i]["user_slug"])
        if Q[2] not in response["submissions"][i]:
            Q3.append("")
        else:   
            Q3.append(response["submissions"][i][Q[2]]["submission_id"])
        if Q[3] not in response["submissions"][i]:
            Q4.append("")
        else:
            Q4.append(response["submissions"][i][Q[3]]["submission_id"])

In [ ]:
cheater = []
DQed = []
for filename in os.listdir("plagiarism_report/"+CONTEST):
    f_path = os.path.join("plagiarism_report/"+CONTEST, filename)
    if os.path.isfile(f_path) and os.path.splitext(f_path)[-1] == ".csv":
        QUESTION = filename[0:2]
        
        with open(f_path, "r") as f:
            rows = list(csv.reader(f))
        
        DQ = 0
        with open("plagiarism_report/"+CONTEST+'/'+filename[:-4]+".md", "w") as md:
            print("| rank | user | "+QUESTION+"   |\n| ---- | ---- | ---- |", file=md)
            
            with open(f_path, "w") as f:
                for i in rows:
                    try:        
                        if QUESTION == "Q4":
                            idx = Q4.index(int(i[2]))
                        elif QUESTION == "Q3":
                            idx = Q3.index(int(i[2]))
                        else:
                            sys.exit()
                    except:
                        try:
                            idx = user.index(i[1])
                        except:
                            idx = -1
                    
                    
                    if idx != -1:
                        cheater.append(user[idx])
                        
                        print(str(idx+1) + "," + user[idx] + "," + i[2], file=f)
                        print("|", idx+1, "| ["+user[idx]+"](https://leetcode.com/u/"+user[idx]+") |", CONTEST_URL+"submissions/detail/"+i[2], "|", file=md)
                    else:
                        cheater.append(i[1])
                        DQed.append(i[1])
                        
                        DQ += 1
                        print("DQ," + i[1] + "," + i[2], file=f)
                        print("| DQ | ["+i[1]+"](https://leetcode.com/u/"+i[1]+") |", CONTEST_URL+"submissions/detail/"+i[2], "|", file=md)
        
        print(filename, "DQ:", DQ, '/', len(rows))

print("DQ:", len(set(DQed)), '/', len(set(cheater)))